### Состав команды:

- Поляков Александр (Aleksandr Polyakov)
- Востриков Алексей (Alexey Vostrikoff)
- Садыков Динар (Dinar_Sadykow)
- Федотов Андрей (Andrei Fedotov)
- Каштанкина Ксения
- Овчинников Алексей (Aleksey)
- Пьянков Алексей
- Лукошко Роберт (Robert)
- Поперечный Богдан (Bogdan2105)

### Описание данных

* train.csv - данные для обучения
* test.csv - данные для подготовки самбита и проверки
* sampleSubmission.csv - пример корректного но бесполезного сабмита
* other.csv - необязательные данные для доп.статистик и прочих извращений (например обучение word2vec-а)

### Описание полей

* id - внутренний идетификатор
* name - название вакансии
* description - текст вакансии
* target - класс заинтересованности

In [4]:
%%time

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score, recall_score, \
                        accuracy_score, classification_report, roc_auc_score, roc_curve

CPU times: user 237 ms, sys: 82.8 ms, total: 320 ms
Wall time: 447 ms


## Шаг 1. Импортируем все в Pandas


In [5]:
%%time
sample_submission = pd.read_csv('sampleSubmission.csv',',')
train_df = pd.read_csv('train.csv','\t')
test_df = pd.read_csv('test.csv','\t')
other_df = pd.read_csv('other.csv','\t')

CPU times: user 16.9 s, sys: 1.7 s, total: 18.6 s
Wall time: 18.6 s


In [6]:
train_df['concat'] = train_df['name']+' '+ train_df['description']
train_df.head()

,id,name,description,target,concat
0,0,Заведующий отделом/секцией в магазин YORK (Уру...,<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) пр...,1,Заведующий отделом/секцией в магазин YORK (Уру...
1,1,Наладчик станков и манипуляторов с ПУ,Обязанности:работа на токарных станках с ЧПУ T...,0,Наладчик станков и манипуляторов с ПУ Обязанно...
2,2,Разработчик С++ (Криптограф),<strong>Требования:</strong> <ul> <li>Опыт про...,0,Разработчик С++ (Криптограф) <strong>Требовани...
3,3,Фрезеровщик,<p>Условия:</p> <ul> <li>На работу вахтовым ме...,0,Фрезеровщик <p>Условия:</p> <ul> <li>На работу...
4,4,Мерчендайзер/продавец-консультант,<p><strong>Компания Палладиум Стандарт - призн...,1,Мерчендайзер/продавец-консультант <p><strong>К...


## Шаг 2. Чистим данные и подготавливаем текст

In [7]:
# Подключаем БьютифулСуп
from bs4 import BeautifulSoup             

# Прогоняем отдельное описание через БьютифулСуп   
example1 = BeautifulSoup(train_df['concat'][0], 'html5lib')  

In [8]:
#Подключаем регулярки, чтобы очистить текст от тегов html
import re
# Делаем найти и заменить на ничто для всего что не похоже на регулярку
letters_only = re.sub("[^а-яёЁ А-Яa-zA-Z]",           # находим всё что не
                      " ",                   # Меняем на пустоту
                      example1.get_text() )  
print (letters_only)

Заведующий отделом секцией в магазин YORK  Уручье  В НОВЫЙ МАГАЗИН YORK  хозтовары  приглашаем на постоянную работу руководителя секции  хозтовары   Обязанности      организация эффективного и культурного обслуживания покупателей организация приемки сдачи товаров на склад консультация покупателей по вопросам  касающимся оказываемых услуг контроль своевременной подачи товаров в торговую секцию проверка качества  сроков годности ТМЦ  наличие маркировок  ценников на товарах организация бесперебойной работы товарной секции и участка склада  закрепленного за ней контроль за сохранностью товаров  торгового оборудования и прочих материальных ценностей инвентаризация товаров    Требования   знание товарной группы знание правил приемки товара опыт работы в розничной торговле в качестве продавца   р    заместителя заведующего секцией отделом ответственность  лидерские качества  опыт управления коллективом      Условия   НОВЫЙ современный магазин розничной торговли хозтоварами хороший коллектив д

In [9]:
lower_case = letters_only.lower()
words1 = lower_case.split()  

In [10]:
import nltk
import pymystem3
mystem = pymystem3.Mystem()
#nltk.download()  # Если NLTK не установлен, то надо раскомментровать этот блок и исполнить

In [11]:
from nltk.corpus import stopwords # Импортируем стоп-слова русскго языка
print( stopwords.words('russian') )

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [12]:
%%time
import time
# Удаляем стоп-слова русского языка из текста words
a = time.clock()
words = [w for w in words1 if not w in stopwords.words('russian')]
b = time.clock()
print('Заняло {} сек'.format(b-a))
print (words)
# Посмоторим что получится, если прогнать мистемом. 
a = time.clock()
words = [mystem.lemmatize(w) for w in words1 if not w in stopwords.words('russian')]
b = time.clock()
print('Заняло {} сек'.format(b-a))
print (words)

Заняло 0.024632999999997907 сек
['заведующий', 'отделом', 'секцией', 'магазин', 'york', 'уручье', 'новый', 'магазин', 'york', 'хозтовары', 'приглашаем', 'постоянную', 'работу', 'руководителя', 'секции', 'хозтовары', 'обязанности', 'организация', 'эффективного', 'культурного', 'обслуживания', 'покупателей', 'организация', 'приемки', 'сдачи', 'товаров', 'склад', 'консультация', 'покупателей', 'вопросам', 'касающимся', 'оказываемых', 'услуг', 'контроль', 'своевременной', 'подачи', 'товаров', 'торговую', 'секцию', 'проверка', 'качества', 'сроков', 'годности', 'тмц', 'наличие', 'маркировок', 'ценников', 'товарах', 'организация', 'бесперебойной', 'работы', 'товарной', 'секции', 'участка', 'склада', 'закрепленного', 'контроль', 'сохранностью', 'товаров', 'торгового', 'оборудования', 'прочих', 'материальных', 'ценностей', 'инвентаризация', 'товаров', 'требования', 'знание', 'товарной', 'группы', 'знание', 'правил', 'приемки', 'товара', 'опыт', 'работы', 'розничной', 'торговле', 'качестве', 'пр

In [13]:
temp = ''
for i in words:
    temp += (i[0]+' ')
temp

'заведующий отдел секция магазин york уручье новый магазин york хозтовары приглашать постоянный работа руководитель секция хозтовары обязанность организация эффективный культурный обслуживание покупатель организация приемка сдача товар склад консультация покупатель вопрос касаться оказывать услуга контроль своевременный подача товар торговый секция проверка качество срок годность тмц наличие маркировка ценник товар организация бесперебойный работа товарный секция участок склад закреплять контроль сохранность товар торговый оборудование прочий материальный ценность инвентаризация товар требование знание товарный группа знание правило приемка товар опыт работа розничный торговля качество продавец р заместитель заведующий секция отдел ответственность лидерский качество опыт управление коллектив условие новый современный магазин розничный торговля хозтовары хороший коллектив достойный зарплата премия соц пакет график работа пн пт плавать место работа ул никифоров уручье '

In [14]:
def vac_to_words( description, Mystem = True ):
    """
    Функция делает преобразование поля description из формата html в формат с отдельными значащими словами
    На вход передается  описание и параметр True/False. Если True, то работает стеммер, иначе нет
    """
    description_text = BeautifulSoup(description, 'html5lib').get_text() #получаем текст      
    letters_only = re.sub('[^а-яёЁ А-Яa-zA-Z]', ' ', description_text) #удаляем всё что не буквы
    words = letters_only.lower().split() #получаем слова в нижнем регистре                             
    stops = set(stopwords.words('russian'))  # формируем set стоп слов русского языка                  
    if Mystem: #удаляем минус фразы и  оставляем либо "стеммы", либо значащие слова
        meaningful_words = [mystem.lemmatize(w) for w in words if not w in stops]
        temp = ''
        for i in meaningful_words:
            temp += (i[0]+' ')
        return temp
    else: 
        meaningful_words = [w for w in words if not w in stops]
        return( ' '.join( meaningful_words ))  # собираем слова в строку с разделителеим пробел 

In [15]:
clean_vac = vac_to_words( train_df['concat'][0])
print(clean_vac)

заведующий отдел секция магазин york уручье новый магазин york хозтовары приглашать постоянный работа руководитель секция хозтовары обязанность организация эффективный культурный обслуживание покупатель организация приемка сдача товар склад консультация покупатель вопрос касаться оказывать услуга контроль своевременный подача товар торговый секция проверка качество срок годность тмц наличие маркировка ценник товар организация бесперебойный работа товарный секция участок склад закреплять контроль сохранность товар торговый оборудование прочий материальный ценность инвентаризация товар требование знание товарный группа знание правило приемка товар опыт работа розничный торговля качество продавец р заместитель заведующий секция отдел ответственность лидерский качество опыт управление коллектив условие новый современный магазин розничный торговля хозтовары хороший коллектив достойный зарплата премия соц пакет график работа пн пт плавать место работа ул никифоров уручье 


In [16]:
# Посмотрим сколько всего описаний есть в train_df
num_vacs = train_df['concat'].size
num_vacs

200000

In [17]:
%%time
clean_train_vacs = [] #делаем пустой лист для записи всех описаний с отдельными словами

print ('Очищаем и собираем нормализованные описания из train_df...\n')

# Пройдем в цикле по всем описаниям
for i in range( 0, num_vacs ):
    
    # Для каждого описания вызываем функцию нормировки текста описания и добавляем нормированный текст в лист
    
    if( (i+1)%10000 == 0 ): #чтобы не было скучно, каждые 10 000 описаний будем печатать развлекательный текст
        print ('Описание {} из {} в обработке... {:.0%}'.format( i+1, num_vacs,(i+1)/num_vacs )) 
        
    clean_train_vacs.append( vac_to_words( train_df['concat'][i]) )
print('Я закончила!!!')

Очищаем и собираем нормализованные описания из train_df...

Описание 10000 из 200000 в обработке... 5%
Описание 20000 из 200000 в обработке... 10%
Описание 30000 из 200000 в обработке... 15%
Описание 40000 из 200000 в обработке... 20%
Описание 50000 из 200000 в обработке... 25%
Описание 60000 из 200000 в обработке... 30%
Описание 70000 из 200000 в обработке... 35%
Описание 80000 из 200000 в обработке... 40%
Описание 90000 из 200000 в обработке... 45%
Описание 100000 из 200000 в обработке... 50%
Описание 110000 из 200000 в обработке... 55%
Описание 120000 из 200000 в обработке... 60%
Описание 130000 из 200000 в обработке... 65%
Описание 140000 из 200000 в обработке... 70%
Описание 150000 из 200000 в обработке... 75%
Описание 160000 из 200000 в обработке... 80%
Описание 170000 из 200000 в обработке... 85%
Описание 180000 из 200000 в обработке... 90%
Описание 190000 из 200000 в обработке... 95%
Описание 200000 из 200000 в обработке... 100%
Я закончила!!!
CPU times: user 19min 14s, sys: 3m

## Создаём свойства через Bag of Words

In [18]:
%%time
print ('Фигачим Bag of words...\n')
from sklearn.feature_extraction.text import CountVectorizer 
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 10000) 


# 5 000 фич считались 20.1 сек
# прогоняем полученный в прошлом пункте лист с текстами через fit_transform() 
train_data_features = vectorizer.fit_transform(clean_train_vacs)

# Переделаем полученные свойства в array 
train_data_features = train_data_features.toarray()

Фигачим Bag of words...

CPU times: user 17.9 s, sys: 5.53 s, total: 23.4 s
Wall time: 24.3 s


In [19]:
train_data_features.shape

(200000, 10000)

## Шаг 3. Решам задачу через модель Random Forest

In [20]:
%%time
#Делаем трейн тест сплит

from sklearn.cross_validation import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve

X_train, X_test, y_train, y_test = train_test_split(train_data_features, train_df['target'], test_size=0.2, random_state=42)
print('X_train ', X_train.shape)
print('X_test ',X_test.shape)
print('y_train ', y_train.shape)
print('y_test ',y_test.shape)


/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


X_train  (160000, 10000)
X_test  (40000, 10000)
y_train  (160000,)
y_test  (40000,)
CPU times: user 7.39 s, sys: 13.5 s, total: 20.9 s
Wall time: 24.1 s


In [21]:
%%time

print ('Обучаем Random Forest...\n')
from sklearn.ensemble import RandomForestClassifier

# Пока делаем 100 деревьев, но в следующих вариантах домашки попробую сделать 50
forest = RandomForestClassifier(n_estimators = 100, n_jobs = -1, random_state = 123) 

forest = forest.fit( X_train, y_train )
print ('Саш, я обучилась!')

Обучаем Random Forest...

Саш, я обучилась!
CPU times: user 1h 12min 4s, sys: 1min 6s, total: 1h 13min 10s
Wall time: 8min 45s


In [22]:
predict = forest.predict(X_test)

In [23]:
%%time

# прошлый результаты были 0.97 0.97 0.97
y_hat = forest.predict_proba(X_test)
print('#'*20,'>>> Random Trees <<<','#'*20,'\n',classification_report(y_test, predict),'\n')
print('ROCAUC = {}'.format( roc_auc_score(y_test, y_hat[:,1])))

#################### >>> Random Trees <<< #################### 
              precision    recall  f1-score   support

          0       0.97      0.96      0.97     21268
          1       0.96      0.97      0.96     18732

avg / total       0.97      0.97      0.97     40000
 

ROCAUC = 0.9928749454155948
CPU times: user 15.6 s, sys: 816 ms, total: 16.4 s
Wall time: 2.94 s


In [24]:

from sklearn.linear_model import LogisticRegression



In [25]:
%%time

logreg = LogisticRegression(random_state = 123, n_jobs = -1)
logreg = logreg.fit( X_train, y_train )
predict = logreg.predict(X_test)
y_hat = logreg.predict_proba(X_test)
print('#'*20,'>>> Logistic Regression <<<','#'*20,'\n',classification_report(y_test, predict),'\n')
print('ROCAUC = {}'.format( roc_auc_score(y_test, y_hat[:,1])))

/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


#################### >>> Logistic Regression <<< #################### 
              precision    recall  f1-score   support

          0       0.96      0.96      0.96     21268
          1       0.95      0.96      0.96     18732

avg / total       0.96      0.96      0.96     40000
 

ROCAUC = 0.9881096008271006
CPU times: user 4min 27s, sys: 28.1 s, total: 4min 55s
Wall time: 1min 39s


## Пробуем стэккинг


In [26]:
from vecstack import stacking

In [27]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import RadiusNeighborsClassifier

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import validation_curve
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [28]:
%%time

models = [
    RandomForestClassifier(n_estimators=100, max_depth=5, random_state=123,  n_jobs = -1),
    RandomForestClassifier(n_estimators=100, max_depth=10, random_state=123, max_features='log2',  n_jobs = -1),
    LogisticRegression(class_weight = 'balanced', random_state = 123, n_jobs = -1),
    LogisticRegression(random_state = 123, n_jobs = -1),
    RandomForestClassifier(n_estimators=100, random_state=123,  n_jobs = -1)
    
]

CPU times: user 137 µs, sys: 446 µs, total: 583 µs
Wall time: 587 µs


In [29]:
def roc_auc_score_cust(y_true, y_hat):
    return roc_auc_score(y_true, y_hat[:,1])

In [30]:
%%time

S_train, S_valid = stacking(models,
                               X_train, y_train, X_test,
                               regression=False,
                               mode='oof_pred_bag', 
                               needs_proba=True,
                               metric=roc_auc_score_cust,
                               n_folds=5,                
                               stratified=True,          
                               shuffle=True,             
                               random_state=123,         
                               verbose=2)                

task:         [classification]
n_classes:    [2]
metric:       [roc_auc_score_cust]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [RandomForestClassifier]
    fold  0:  [0.97544600]
    fold  1:  [0.97541940]
    fold  2:  [0.97426876]
    fold  3:  [0.97589266]
    fold  4:  [0.97606548]
    ----
    MEAN:     [0.97541846] + [0.00062707]
    FULL:     [0.97541951]

model  1:     [RandomForestClassifier]
    fold  0:  [0.96592820]
    fold  1:  [0.96898151]
    fold  2:  [0.96645308]
    fold  3:  [0.96404066]
    fold  4:  [0.96682817]
    ----
    MEAN:     [0.96644632] + [0.00158926]
    FULL:     [0.96642698]

model  2:     [LogisticRegression]


/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


    fold  0:  [0.98778995]


/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


    fold  1:  [0.98758644]


...

    fold  2:  [0.98732591]


/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


    fold  3:  [0.98811598]


/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


    fold  4:  [0.98737497]
    ----
    MEAN:     [0.98764600] + [0.00029037]
    FULL:     [0.98764410]

model  4:     [RandomForestClassifier]
    fold  0:  [0.99214268]
    fold  1:  [0.99198643]
    fold  2:  [0.99209288]
    fold  3:  [0.99279263]
    fold  4:  [0.99204606]
    ----
    MEAN:     [0.99221214] + [0.00029480]
    FULL:     [0.99221093]

CPU times: user 6h 2min 8s, sys: 15min 45s, total: 6h 17min 54s
Wall time: 1h 1min 35s


In [31]:
%%time

for model in models:
    model.fit(X_train, y_train)
    y_hat = model.predict_proba(X_test)
    score = roc_auc_score(y_test, y_hat[:, 1])
    print('{}: {}'.format(model.__str__().split('(')[0], score))

RandomForestClassifier: 0.9756715202157986
RandomForestClassifier: 0.9671345704339335


/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


LogisticRegression: 0.9880950197174555


/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


LogisticRegression: 0.9881096008271006
RandomForestClassifier: 0.9928749429055053
CPU times: user 1h 40min 23s, sys: 1min 35s, total: 1h 41min 58s
Wall time: 12min 7s


In [33]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
last_model = GradientBoostingClassifier(n_estimators=300, max_depth=3,
                                       learning_rate=0.01, 
                                       random_state=123)

CPU times: user 47 µs, sys: 112 µs, total: 159 µs
Wall time: 164 µs


In [35]:
%%time
last_model.fit(S_train, y_train)
y_hat = last_model.predict_proba(S_valid)
roc_auc_score(y_test, y_hat[:, 1])

CPU times: user 43.7 s, sys: 257 ms, total: 44 s
Wall time: 44 s


In [36]:
roc_auc_score(y_test, y_hat[:, 1])

0.99384697253693055

In [44]:
%%time
# прошлый результаты были 0.97 0.97 0.97
predict = last_model.predict(S_valid)

print('#'*20,'>>> Stacking <<<','#'*20,'\n',classification_report(y_test, predict),'\n')

#################### >>> Stacking <<< #################### 
              precision    recall  f1-score   support

          0       0.98      0.96      0.97     21268
          1       0.96      0.97      0.97     18732

avg / total       0.97      0.97      0.97     40000
 

CPU times: user 169 ms, sys: 1.62 ms, total: 170 ms
Wall time: 169 ms


In [41]:
S_train.shape

(160000, 10)

In [42]:
S_valid.shape

(40000, 10)

### Шаг 4. Готовим файл к отправке

In [45]:
test_df['concat'] = test_df['name']+' '+ test_df['description']

test_df.shape

(170179, 4)

In [46]:
num_test_vacs = len(test_df['concat'])

In [47]:
%%time
clean_test_vacs = []
print ('Очищаем и собираем нормализованные описания из test_df...\n')
for i in range(0,num_test_vacs):
    if( (i+1) % 10000 == 0 ):
        print ('Описание {} из {} в обработке... {:.0%}'.format( i+1, num_test_vacs, (i+1)/num_test_vacs )) 
    clean_test_vacs.append( vac_to_words( test_df['concat'][i] ) )
print('Я закончила!!!')

Очищаем и собираем нормализованные описания из test_df...

Описание 10000 из 170179 в обработке... 6%
Описание 20000 из 170179 в обработке... 12%
Описание 30000 из 170179 в обработке... 18%
Описание 40000 из 170179 в обработке... 24%
Описание 50000 из 170179 в обработке... 29%
Описание 60000 из 170179 в обработке... 35%
Описание 70000 из 170179 в обработке... 41%
Описание 80000 из 170179 в обработке... 47%
Описание 90000 из 170179 в обработке... 53%
Описание 100000 из 170179 в обработке... 59%
Описание 110000 из 170179 в обработке... 65%
Описание 120000 из 170179 в обработке... 71%
Описание 130000 из 170179 в обработке... 76%
Описание 140000 из 170179 в обработке... 82%
Описание 150000 из 170179 в обработке... 88%
Описание 160000 из 170179 в обработке... 94%
Описание 170000 из 170179 в обработке... 100%
Я закончила!!!
CPU times: user 15min 41s, sys: 3min, total: 18min 41s
Wall time: 30min 56s


In [48]:
%%time
# Фигачим bag of words по тестовым данным

test_data_features = vectorizer.transform(clean_test_vacs)
test_data_features = test_data_features.toarray()

CPU times: user 16 s, sys: 4.46 s, total: 20.4 s
Wall time: 20.7 s


In [49]:
%%time
# просто содрал с лекции Эдуарда. Если метод сработает - 
# надо одолеть Эдуарда вопросами про параметры, которые мы тут передаём
S_train, S_valid = stacking(models,
                               train_data_features, train_df['target'], test_data_features,
                               regression=False,
                               mode='oof_pred_bag', 
                               needs_proba=True,
                               metric=roc_auc_score_cust,
                               n_folds=5,                
                               stratified=True,          
                               shuffle=True,             
                               random_state=123,         
                               verbose=2)   

task:         [classification]
n_classes:    [2]
metric:       [roc_auc_score_cust]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [RandomForestClassifier]
    fold  0:  [0.97477843]
    fold  1:  [0.97510637]
    fold  2:  [0.97620028]
    fold  3:  [0.97543412]
    fold  4:  [0.97602449]
    ----
    MEAN:     [0.97550874] + [0.00053760]
    FULL:     [0.97550135]

model  1:     [RandomForestClassifier]
    fold  0:  [0.96724913]
    fold  1:  [0.96517289]
    fold  2:  [0.96605382]
    fold  3:  [0.96866251]
    fold  4:  [0.96382720]
    ----
    MEAN:     [0.96619311] + [0.00166559]
    FULL:     [0.96620111]

model  2:     [LogisticRegression]


/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


    fold  0:  [0.98843013]


/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


    fold  1:  [0.98776716]


...

    fold  2:  [0.98859456]


/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


    fold  3:  [0.98811866]


/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


    fold  4:  [0.98845646]
    ----
    MEAN:     [0.98828667] + [0.00029208]
    FULL:     [0.98828939]

model  4:     [RandomForestClassifier]
    fold  0:  [0.99258269]
    fold  1:  [0.99203233]
    fold  2:  [0.99281011]
    fold  3:  [0.99297267]
    fold  4:  [0.99299781]
    ----
    MEAN:     [0.99267912] + [0.00035568]
    FULL:     [0.99267796]

CPU times: user 7h 57min 43s, sys: 29min 38s, total: 8h 27min 22s
Wall time: 1h 39min 17s


In [50]:
last_model = GradientBoostingClassifier(n_estimators=300, max_depth=3,
                                       learning_rate=0.01, 
                                       random_state=123)

In [51]:
last_model.fit(S_train, train_df['target'])

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=300,
              presort='auto', random_state=123, subsample=1.0, verbose=0,
              warm_start=False)

In [52]:
result_test = last_model.predict(S_valid)

In [53]:
# Копируем результат в датафрейм с полями id и target 
# предыдущий реузльтат был таким http://take.ms/cVRK4
output = pd.DataFrame( data={'id':test_df['id'], 'target':result_test} )
output.head()

,id,target
0,200000,1
1,200001,1
2,200002,1
3,200003,1
4,200004,0


In [54]:
output.to_csv( "Stacking_10000mystem-and-names.csv", sep=',', encoding='utf-8',index=False )